In [77]:
import pandas as pd
import numpy as np
rating_df = pd.read_csv("../data/raw/ml-100k/u.data", sep="\t", index_col=False, names=["userID", "itemID", "rating"])
rating_df.head(10)

C:\Users\freid\AppData\Local\Temp\ipykernel_11632\648380794.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  rating_df = pd.read_csv("../data/raw/ml-100k/u.data", sep="\t", index_col=False, names=["userID", "itemID", "rating"])


,userID,itemID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
5,298,474,4
6,115,265,2
7,253,465,5
8,305,451,3
9,6,86,3


In [78]:
# From u.info
# 943 users
# 1682 items
# 100000 ratings
user_item_matrix = np.zeros((943,1682), dtype=float)
for index, r in rating_df.iterrows():
    user_item_matrix[r["userID"]-1][r["itemID"]-1] = r["rating"]
    
print(user_item_matrix)

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [79]:
import numpy as np

fraction = 0.85  # Adjust the fraction

np.random.shuffle(user_item_matrix)

split_index = int(len(user_item_matrix) * fraction)

train_user_item_matrix = user_item_matrix[:split_index]
test_user_item_matrix = user_item_matrix[split_index:]

In [80]:
train_user_item_matrix[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.]])

In [81]:
def als(matrix, latent_features, iterations=10, lambda_reg=0.01):
    """
    Alternating Least Squares (ALS) matrix factorization.

    Parameters:
    - matrix: User-Item interaction matrix to factorize.
    - latent_features: Number of latent features.
    - iterations: Number of iterations.
    - lambda_reg: Regularization parameter.

    Returns:
    - user_matrix: Factorized user matrix.
    - item_matrix: Factorized item matrix.
    """

    num_users, num_items = matrix.shape

    # Initialize user and item matrices with random values
    user_matrix = np.random.rand(num_users, latent_features)
    item_matrix = np.random.rand(num_items, latent_features)

    # ALS iterations
    for _ in range(iterations):
        # Update user matrix while fixing item matrix
        for i in range(num_users):
            matrix_i = matrix[i, :].reshape((1, -1))  # Extract user's ratings
            item_matrix_transpose = item_matrix.T
            user_matrix[i, :] = np.linalg.solve(
                np.dot(item_matrix_transpose, item_matrix) + lambda_reg * np.eye(latent_features),
                np.dot(item_matrix_transpose, matrix_i.T)
            ).flatten()

        # Update item matrix while fixing user matrix
        for j in range(num_items):
            matrix_j = matrix[:, j].reshape((-1, 1))  # Extract item's ratings
            user_matrix_transpose = user_matrix.T
            item_matrix[j, :] = np.linalg.solve(
                np.dot(user_matrix_transpose, user_matrix) + lambda_reg * np.eye(latent_features),
                np.dot(user_matrix_transpose, matrix_j)
            ).flatten()

    return user_matrix, item_matrix

# Example usage:
# Assuming 'user_item_matrix' is your user-item interaction matrix
latent_features = 70
iterations = 10
lambda_reg = 0.000001

user_matrix, item_matrix = als(train_user_item_matrix, latent_features, iterations, lambda_reg)

# Predictions can be obtained by multiplying the factorized matrices
predictions = np.dot(user_matrix, item_matrix.T)
print("Predictions:\n", predictions)

Predictions:
 [[-8.59726436e-02  6.90809298e-02  1.25432370e-02 ...  3.13618089e-02
   5.61344715e-03 -2.00547376e-02]
 [ 2.38266370e+00  1.00161118e+00  2.95527665e-01 ... -2.21234193e-03
  -5.77816510e-03  6.71655042e-03]
 [ 2.97739289e+00  1.12167586e-01  4.24010302e-01 ... -6.71979517e-04
  -2.52111770e-02  7.13100985e-03]
 ...
 [ 1.57577147e+00 -1.41119783e-01  2.42876223e-01 ... -9.72922132e-03
  -2.07789124e-02 -1.06994261e-02]
 [ 2.57327370e-01 -9.64780964e-02  8.62849777e-02 ...  2.00987848e-02
  -1.92475913e-02 -1.26805127e-03]
 [-5.59489575e-01 -7.35788636e-01  9.81782014e-01 ...  6.02810242e-03
  -3.33947500e-02  4.34429586e-02]]


In [84]:
print(np.mean((predictions - train_user_item_matrix) ** 2))

0.29359786528677945


In [85]:
import numpy as np

# Example matrix A
A = item_matrix.T

# Compute the right inverse using the pseudo-inverse
A_plus = np.linalg.pinv(A)

# Verify the right inverse property
identity_matrix = A @ A_plus
I = np.eye(identity_matrix.shape[0])
print("Right Inverse:\n", A_plus)
print("Verification (A * A^+):\n", np.mean((identity_matrix - I) ** 2))


Right Inverse:
 [[ 9.18673032e-04  2.09945003e-03  2.86402029e-03 ... -2.46221555e-03
  -2.65053830e-04  1.89654445e-03]
 [-1.60686684e-04 -2.82535972e-04  8.38136717e-04 ...  9.34875745e-04
  -7.92699307e-04 -1.00329352e-04]
 [-1.75538874e-03 -1.01639667e-03  1.39263313e-03 ...  7.53378685e-04
  -2.18741926e-05 -7.26201232e-04]
 ...
 [-4.47214986e-05 -4.03808601e-07  4.53196086e-05 ...  3.89841024e-05
  -1.35874361e-05  3.02096818e-05]
 [-1.20521923e-04  5.39029664e-05  1.29393499e-04 ... -5.36012567e-05
  -5.71493752e-05 -3.00501180e-05]
 [ 9.29388644e-05  5.67127672e-05  5.20285242e-05 ...  6.07513393e-05
   3.02778417e-05  5.55565516e-05]]
Verification (A * A^+):
 1.656361917794222e-31


In [185]:
def predict(user_ratings):
    user_latent_features = user_ratings @ A_plus
    return user_latent_features

n_start_rating = 15
X_test = np.empty((len(test_user_item_matrix), 1682))
y_test = []
for i in range(len(test_user_item_matrix)):
    
    idxs = np.nonzero(test_user_item_matrix[i])
    idxs = np.array(idxs)[0]
    X_idxs = np.random.choice(idxs, size=n_start_rating)
    X = np.zeros(1682)
    X[X_idxs] = test_user_item_matrix[i][X_idxs]
    X_test[i] = X
    y_test.append(sorted(list(idxs), key=lambda x: test_user_item_matrix[i][x], reverse=True))
print(np.nonzero(X_test[0]))
print(y_test[0])


(array([  8,  63,  70, 125, 236, 294, 299, 322, 470, 471, 594, 844, 865,
       933], dtype=int64),)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (142,) + inhomogeneous part.

In [198]:
from sklearn.neighbors import NearestNeighbors

from metrics.avg_f1_score import average_f1_score
from metrics.NDCG import ndcg_metric
# Fit the NearestNeighbors model with cosine similarity
nbrs = NearestNeighbors(n_neighbors=3, metric='minkowski')
nbrs.fit(user_matrix)
K = 20
# Example usage
avg_ndcg = 0
s= 0 
for i in range(len(X_test)):
    q = X_test[i]
    distances, indices = nbrs.kneighbors([q @ A_plus])
    non_zero_columns = np.where(np.any(user_item_matrix[indices[0]] > 0, axis=0))[0]
    rating_sum_for_film = np.sum(user_item_matrix[:, non_zero_columns], axis=0)
    non_zero_rating_for_film = np.sum(user_item_matrix[:, non_zero_columns] > 0, axis=0)
    avg_rating_film = rating_sum_for_film / non_zero_rating_for_film
    films = zip(list(non_zero_columns), list(avg_rating_film))
    films = [i[0] for i in sorted(films, key=lambda x: x[1], reverse=True)][:K]
    best_sc = np.sort(test_user_item_matrix[i][test_user_item_matrix[i] > 0])[::-1]
    s+=len(set(y_test[i]) & set(films)) / len(y_test[i])
    pred = test_user_item_matrix[i][films]
    avg_ndcg += ndcg_metric(pred, list(best_sc[:K]))
print(s / len(X_test))
print(avg_ndcg / len(X_test))

ImportError: cannot import name 'ndcg_metric' from 'metrics.NDCG' (C:\Users\freid\PycharmProjects\movie-recommeder-system\metrics\NDCG.py)